<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/analyse_LA_mobility_poverty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [1]:
# Install required dependancies
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 73kB/s 
     |████████████████████████████████| 204kB 36.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=794cb2baf7f0ec362ad6309f04a3ce07bff92791a215809f5cdf5f1c3f010661
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [2]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [3]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [4]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [5]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Demographic Data Chicago
!ls drive/MyDrive/big-data-project/data/clean-data/la/poverty_and_white_percentage.csv

!ls drive/MyDrive/big-data-project/data/clean-data/la/social/2020/



drive/MyDrive/big-data-project/data/clean-data/la/poverty_and_white_percentage.csv
2020-01-01-social-distancing.csv  2020-07-02-social-distancing.csv
2020-01-02-social-distancing.csv  2020-07-03-social-distancing.csv
2020-01-03-social-distancing.csv  2020-07-04-social-distancing.csv
2020-01-04-social-distancing.csv  2020-07-05-social-distancing.csv
2020-01-05-social-distancing.csv  2020-07-06-social-distancing.csv
2020-01-06-social-distancing.csv  2020-07-07-social-distancing.csv
2020-01-07-social-distancing.csv  2020-07-08-social-distancing.csv
2020-01-08-social-distancing.csv  2020-07-09-social-distancing.csv
2020-01-09-social-distancing.csv  2020-07-10-social-distancing.csv
2020-01-10-social-distancing.csv  2020-07-11-social-distancing.csv
2020-01-11-social-distancing.csv  2020-07-12-social-distancing.csv
2020-01-12-social-distancing.csv  2020-07-13-social-distancing.csv
2020-01-13-social-distancing.csv  2020-07-14-social-distancing.csv
2020-01-14-social-distancing.csv  2020-07-15-s

In [8]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'la'

In [9]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [10]:
# View schema
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [49]:
# # Take small sample of data to experiment with
df_soc = df_soc.limit(100)
df_soc.show()

+------------+--------------------+------------+----------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+
|         cbg|    date_range_start|device_count|completely_home_device_count|completely_home_percentage|completely_home_percentage|completely_home_percentage|completely_home_percentage|completely_home_percentage|completely_home_percentage|completely_home_percentage|
+------------+--------------------+------------+----------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+--------------------------+
|060371831013|2020-02-07T00:00:...|          45|                           6|        13.333333333333334|        13.333333333333334|        13.333333333333334|        13.333333333333334|        13.333

In [50]:
df_soc.createOrReplaceTempView('T')
df_soc = spark.sql('SELECT cbg, date_range_start, device_count, completely_home_device_count FROM T')
df_soc.show()

+------------+--------------------+------------+----------------------------+
|         cbg|    date_range_start|device_count|completely_home_device_count|
+------------+--------------------+------------+----------------------------+
|060371831013|2020-02-07T00:00:...|          45|                           6|
|060374034022|2020-02-07T00:00:...|         155|                          24|
|060375326041|2020-02-07T00:00:...|          31|                           4|
|060375544031|2020-02-07T00:00:...|         130|                          27|
|060375718003|2020-02-07T00:00:...|          35|                          10|
|060590018024|2020-02-07T00:00:...|          38|                          13|
|060590320463|2020-02-07T00:00:...|         184|                          24|
|060590626281|2020-02-07T00:00:...|         108|                          32|
|060371374011|2020-02-07T00:00:...|          72|                          30|
|060372398012|2020-02-07T00:00:...|          41|                

In [51]:
# View schema
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)



In [54]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc.show()

+------------+--------------------+------------+----------------------------+--------------------------+
|         cbg|    date_range_start|device_count|completely_home_device_count|completely_home_percentage|
+------------+--------------------+------------+----------------------------+--------------------------+
|060371831013|2020-02-07T00:00:...|          45|                           6|        13.333333333333334|
|060374034022|2020-02-07T00:00:...|         155|                          24|        15.483870967741936|
|060375326041|2020-02-07T00:00:...|          31|                           4|        12.903225806451612|
|060375544031|2020-02-07T00:00:...|         130|                          27|         20.76923076923077|
|060375718003|2020-02-07T00:00:...|          35|                          10|         28.57142857142857|
|060590018024|2020-02-07T00:00:...|          38|                          13|         34.21052631578947|
|060590320463|2020-02-07T00:00:...|         184|       

In [55]:
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- completely_home_percentage: double (nullable = true)



In [56]:
df_soc.createOrReplaceTempView('t')
# get mean percentage for each cbg
df_soc = spark.sql('SELECT cbg, AVG(completely_home_percentage) AS mean_completely_home_percentage FROM t GROUP BY cbg')
df_soc.createOrReplaceTempView('t')
df_soc.show()

+------------+-------------------------------+
|         cbg|mean_completely_home_percentage|
+------------+-------------------------------+
|060371831013|             13.333333333333334|
|060374034022|             15.483870967741936|
|060375326041|             12.903225806451612|
|060375544031|              20.76923076923077|
|060375718003|              28.57142857142857|
|060590018024|              34.21052631578947|
|060590320463|             13.043478260869565|
|060590626281|             29.629629629629626|
|060371374011|              41.66666666666667|
|060372398012|              36.58536585365854|
|060373009011|             24.390243902439025|
|060375418021|             24.489795918367346|
|060375431003|             13.513513513513514|
|060376012021|                           25.0|
|060379800211|                            6.0|
|060590871034|             34.090909090909086|
|060372713005|                           10.0|
|060372766041|              17.51824817518248|
|060374334012

In [57]:
# Read poverty data
# Read data into dataframe
df_pov = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'poverty_and_white_percentage.csv'))
df_pov.createOrReplaceTempView('d')
df_pov.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|060373104003|     1286|17.884914463452567| 69.51788491446345|
|060590865023|     2545|22.789783889980352| 85.34381139489194|
|060376510024|     1285|               0.0|42.490272373540854|
|060376511011|     2708| 7.745398773006134| 50.70162481536189|
|060590994023|      672|12.969283276450511| 38.54166666666667|
|060590995094|      342| 17.83625730994152| 77.48538011695906|
|060376511012|     2483|2.6178010471204187|50.302053966975436|
|060376512011|      859|2.8846153846153846| 82.42142025611176|
|060376512012|     1179| 1.441899915182358|  74.1306191687871|
|060376512014|      989|2.4266936299292214| 70.57633973710819|
|060376512211|     3373|  9.95684340320592| 40.61666172546695|
|060590995092|      938| 5.756929637526652|  84.9680170575693|
|060590995095|      576|14.583333333333334|           7

In [58]:
# join mobility and poverty
result = spark.sql('SELECT d.*, t.mean_completely_home_percentage FROM t INNER JOIN d ON t.cbg = d.cbg')
result.createOrReplaceTempView('t')
result.show()

+------------+---------+------------------+------------------+-------------------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|mean_completely_home_percentage|
+------------+---------+------------------+------------------+-------------------------------+
|060375024011|     1243|3.2984714400643607| 70.39420756234915|                           18.0|
|060375718003|      837|3.1537450722733245|31.780167264038234|              28.57142857142857|
|060375322002|     1976|25.202429149797574| 74.08906882591093|              30.88235294117647|
|060375017003|      928| 7.543103448275862| 74.67672413793103|             15.789473684210526|
|060372766041|     2280| 21.44736842105263| 53.90350877192982|              17.51824817518248|
|060371254022|      576|19.444444444444446| 69.96527777777779|                           37.5|
|060372972011|     2611|14.132516277288396|61.700497893527384|             14.772727272727273|
|060375021001|     1042| 4.838709677419355| 65.930

In [59]:
# Count number of rows
spark.sql('SELECT COUNT(cbg) FROM t').show()

+----------+
|count(cbg)|
+----------+
|       100|
+----------+



In [60]:
# sort by poverty
result = spark.sql('SELECT * FROM t ORDER BY CAST(poverty_percentage AS float) DESC')
result.createOrReplaceTempView('t')
result.show()

+------------+---------+------------------+------------------+-------------------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|mean_completely_home_percentage|
+------------+---------+------------------+------------------+-------------------------------+
|060375403003|      440|              52.5| 46.13636363636363|                           40.0|
|060375762002|     1060|51.886792452830186| 42.16981132075472|                          31.25|
|060374081381|     1699| 43.84932313125368|53.325485579752794|             27.586206896551722|
|060372406003|     1416|40.324858757062145|14.336158192090394|             19.047619047619047|
|060375313013|     1292|39.898132427843805|43.575851393188856|             26.666666666666668|
|060375344041|     1616| 39.29455445544555| 64.72772277227723|               32.6530612244898|
|060372398012|     1766|36.773090079817564|33.691959229898075|              36.58536585365854|
|060372037101|     1629| 36.37484586929716| 71.945

In [ ]:
# group by poverty range


In [61]:
result_df = result.toPandas()
result_df

,cbg,pop_total,poverty_percentage,perc_whiteonly,mean_completely_home_percentage
0,060375403003,440,52.5,46.13636363636363,40.000000
1,060375762002,1060,51.886792452830186,42.16981132075472,31.250000
2,060374081381,1699,43.84932313125368,53.325485579752794,27.586207
3,060372406003,1416,40.324858757062145,14.336158192090394,19.047619
4,060375313013,1292,39.898132427843805,43.575851393188856,26.666667
...,...,...,...,...,...
95,060590320121,1790,0.0,84.1340782122905,18.811881
96,060379800211,0,0.0,0.0,6.000000
97,060379200354,895,0.0,80.44692737430168,32.835821
98,060372690001,606,0.0,86.79867986798679,16.279070


In [62]:
# plot bar graph
alt.Chart(result_df).mark_point().encode(
    alt.X('poverty_percentage:Q'),
    alt.Y('mean_completely_home_percentage:Q'),
    tooltip=['cbg','poverty_percentage','mean_completely_home_percentage']
).properties(width=400, height=400).interactive()

alt.Chart(...)

In [63]:
temp = result_df.groupby(pd.cut(result_df['poverty_percentage'].astype(float), np.arange(0,110, 10))).mean().reset_index()

In [64]:
temp['poverty_percentage'] = temp['poverty_percentage'].astype(str)

In [65]:
temp['color'] = ['#45a0d1' if x>30 else '#ff4833' for x in temp['mean_completely_home_percentage']]
temp

,poverty_percentage,mean_completely_home_percentage,color
0,"(0, 10]",19.179780,#ff4833
1,"(10, 20]",23.754898,#ff4833
2,"(20, 30]",24.839016,#ff4833
3,"(30, 40]",28.597013,#ff4833
4,"(40, 50]",23.316913,#ff4833
5,"(50, 60]",35.625000,#45a0d1
6,"(60, 70]",NaN,#ff4833
7,"(70, 80]",NaN,#ff4833
8,"(80, 90]",NaN,#ff4833
9,"(90, 100]",NaN,#ff4833


In [66]:
# 
alt.Chart(temp).mark_point().encode(
    alt.X('poverty_percentage:N'),
    alt.Y('mean_completely_home_percentage'),
    color = alt.Color('color',scale=None),
    tooltip=[alt.Tooltip('poverty_percentage'),
             alt.Tooltip('mean_completely_home_percentage')]
).properties(width=400, height=400).interactive()

alt.Chart(...)

In [67]:
result_df['poverty_percentage'].astype(float).dtype

dtype('float64')